In [1]:
import os
import pandas as pd
import json
from pathlib import Path
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

with open(Path("~/pg_db_config.json").expanduser(), "r") as f:
    db_params = json.load(f)

engine = create_engine(f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}")

def run_query(query):
    df = pd.read_sql(query, engine)
    return df


In [2]:
datasets = ["2016", "2020", "2024"]
dfs = {}

for year in datasets:
    query = f"SELECT * FROM analysis.nc_pct_shape_results_polling_locs_voter_reg_{year};"
    dfs[f"df_{year}"] = run_query(query).drop(columns=['geom'])

    
df_2016, df_2020, df_2024 = dfs["df_2016"], dfs["df_2020"], dfs["df_2024"]


In [3]:
# Standardize column names
rename_2016 = {"g16_total_votes": "total_votes_2016", "total_voters": "total_registered_2016"}
rename_2020 = {"g20_total_votes": "total_votes_2020", "total_voters": "total_registered_2020"}
rename_2024 = {"total_votes": "total_votes_2024", "total_voters": "total_registered_2024"}

df_2016.rename(columns=rename_2016, inplace=True)
df_2020.rename(columns=rename_2020, inplace=True)
df_2024.rename(columns=rename_2024, inplace=True)

# Select relevant columns
common_merge_cols = ["county_nam", "precinct_name", "prec_id"]
demographic_cols = ["dem_pct", "gop_pct", "black_pct", "white_pct", "hl_pct", "fem_pct", "male_pct", 
                    "18_25_pct", "26_40_pct", "41_65_pct", "65plus_pct"]

df_2016 = df_2016[common_merge_cols + list(rename_2016.values()) + demographic_cols]
df_2020 = df_2020[common_merge_cols + list(rename_2020.values()) + demographic_cols]
df_2024 = df_2024[common_merge_cols + list(rename_2024.values()) + demographic_cols]

# Merge datasets
df_merged = df_2016.merge(df_2020, on=common_merge_cols, how="outer", suffixes=("_2016", "_2020")) \
                    .merge(df_2024, on=common_merge_cols, how="outer")

# Compute turnout rates
df_merged["turnout_2016"] = df_merged["total_votes_2016"] / df_merged["total_registered_2016"]
df_merged["turnout_2020"] = df_merged["total_votes_2020"] / df_merged["total_registered_2020"]
df_merged["turnout_2024"] = df_merged["total_votes_2024"] / df_merged["total_registered_2024"]

# Compute voter registration growth
df_merged["growth_rate_precinct"] = (df_merged["total_registered_2024"] - df_merged["total_registered_2020"]) / df_merged["total_registered_2020"]

# Compute county-level voter registration growth
county_growth = df_merged.groupby("county_nam")[["total_registered_2020", "total_registered_2024"]].sum()
county_growth["growth_rate_county"] = (county_growth["total_registered_2024"] - county_growth["total_registered_2020"]) / county_growth["total_registered_2020"]
df_merged = df_merged.merge(county_growth[["growth_rate_county"]], on="county_nam", how="left")

# Compute turnout loss
df_merged["turnout_change_2020_2024"] = df_merged["turnout_2024"] - df_merged["turnout_2020"]

# Train XGBoost Model
X = df_merged[["growth_rate_precinct", "growth_rate_county"] + demographic_cols].fillna(0)
y = df_merged["turnout_change_2020_2024"].fillna(0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_model = XGBRegressor(objective="reg:squarederror", random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

# Train Random Forest with GridSearchCV
param_grid = {"n_estimators": [50, 100, 200], "max_depth": [None, 10, 20], "min_samples_split": [2, 5, 10]}
grid_search_rf = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=3, scoring="r2")
grid_search_rf.fit(X_train, y_train)

rf_best_model = grid_search_rf.best_estimator_
y_pred_rf_optimized = rf_best_model.predict(X_test)

# Compare Model Performance
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)
mse_rf = mean_squared_error(y_test, y_pred_rf_optimized)
r2_rf = r2_score(y_test, y_pred_rf_optimized)

print("XGBoost MSE:", mse_xgb, "XGBoost R²:", r2_xgb)
print("Optimized RF MSE:", mse_rf, "Optimized RF R²:", r2_rf)

# df_merged.to_csv("turnout_impact_analysis.csv", index=False)


XGBoost MSE: 0.0035723570009991226 XGBoost R²: 0.951219008796018
Optimized RF MSE: 0.003007469161276806 Optimized RF R²: 0.9589326244097497


In [7]:
numeric_cols = df_merged.select_dtypes(include=["number"])

correlation_matrix = numeric_cols.corr()

correlation_matrix["turnout_change_2020_2024"].sort_values(ascending=False)

turnout_change_2020_2024    1.000000
turnout_2024                0.999905
turnout_2016                0.865373
white_pct                   0.747209
white_pct_2020              0.744892
white_pct_2016              0.740641
gop_pct_2016                0.650806
gop_pct_2020                0.625688
male_pct                    0.602298
gop_pct                     0.586536
male_pct_2016               0.539555
male_pct_2020               0.521759
41_65_pct_2016              0.473089
41_65_pct_2020              0.452381
41_65_pct                   0.444256
65plus_pct                  0.440114
65plus_pct_2020             0.430492
65plus_pct_2016             0.406819
growth_rate_precinct        0.230995
turnout_2020                0.161073
total_votes_2024            0.141772
growth_rate_county          0.120275
total_votes_2020            0.076783
total_votes_2016            0.068868
fem_pct                    -0.002652
total_registered_2024      -0.043954
total_registered_2020      -0.074967
t